In [ ]:
# 1 Library setup
import os
import glob
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge


DATA_DIR = "data"
OUT_DIR = "icu_batches"
MERGED_PATH = "icu_merged.parquet"

BIN_HOURS = 12
BATCH_SIZE = 3000

os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
# 2 Load data and setup metadata

icustays = pd.read_csv(f"{DATA_DIR}/ICUSTAYS.csv")
admissions = pd.read_csv(f"{DATA_DIR}/ADMISSIONS.csv")
patients = pd.read_csv(f"{DATA_DIR}/PATIENTS.csv")

icu_meta = (
    icustays
    .merge(admissions, on=["subject_id", "hadm_id"], how="left")
    .merge(patients, on="subject_id", how="left")
)

icu_meta = icu_meta[[
    "subject_id",
    "hadm_id",
    "icustay_id",
    "intime",
    "outtime"
]]

icu_meta["intime"] = pd.to_datetime(icu_meta["intime"])
icu_meta["outtime"] = pd.to_datetime(icu_meta["outtime"])


In [51]:
# 3 create structure for bins

def build_base_grid(batch_meta):
    rows = []
    for _, r in batch_meta.iterrows():
        n_bins = int(
            np.ceil(
                (r.outtime - r.intime).total_seconds()
                / (BIN_HOURS * 3600)
            )
        )
        for b in range(n_bins):
            rows.append({
                "icustay_id": r.icustay_id,
                "bin": b
            })
    return pd.DataFrame(rows)

In [99]:
# 4 obtain desired vitals info from chartevents

'''VITAL_ITEMIDS = {
    "heart_rate": [211, 220045],
    "mean_bp": [456, 220052],
    "spo2": [646, 220277],
    "resp_rate": [618, 220210],
    "temperature": [223761, 678]
}'''
VITAL_ITEMIDS = {
    # Heart rate
    "heart_rate": [211, 220045],

    # Systolic BP
    "sbp": [51, 442, 455, 6701, 220179],

    # Diastolic BP
    "dbp": [8368, 8440, 8441, 8555, 220180],

    # Respiratory rate
    "resp_rate": [618, 220210],

    # Oxygen saturation
    "spo2": [646, 220277],

    # Temperature (Fahrenheit)
    "temperature": [223761, 678],

    # Glasgow Coma Scale (total)
    "gcs": [198, 226755],

    # FiO2 (fraction, NOT %)
    "fio2": [190, 223835],

    # O2 flow rate
    "o2_flow": [470, 471, 223834],
}
ALL_VITAL_ITEMIDS = sum(VITAL_ITEMIDS.values(), [])

def build_vitals(batch_icustays, icu_meta):
    chunks = pd.read_csv(
        f"{DATA_DIR}/CHARTEVENTS.csv",
        usecols=["icustay_id", "itemid", "charttime", "valuenum"],
        chunksize=5_000_000
    )

    out = []

    for df in chunks:
        df = df[df.icustay_id.isin(batch_icustays)]
        df = df[df.itemid.isin(ALL_VITAL_ITEMIDS)]

        if df.empty:
            continue

        df["charttime"] = pd.to_datetime(df["charttime"])

        df = df.merge(
            icu_meta[["icustay_id", "intime"]],
            on="icustay_id",
            how="left"
        )

        df["bin"] = (
            (df.charttime - df.intime)
            .dt.total_seconds() // (BIN_HOURS * 3600)
        ).astype(int)

        df = (
            df.groupby(["icustay_id", "bin", "itemid"])["valuenum"]
            .mean()
            .reset_index()
        )

        out.append(df)

    df = pd.concat(out, ignore_index=True)

    # Pivot ITEMIDs → named columns
    features = {}
    for name, ids in VITAL_ITEMIDS.items():
        features[name] = (
            df[df.itemid.isin(ids)]
            .groupby(["icustay_id", "bin"])["valuenum"]
            .mean()
        )

    vitals = pd.concat(features, axis=1).reset_index()
    return vitals


In [100]:
# 5 obtain desired lab features from labevents

'''LAB_ITEMIDS = {
    "anion gap": [50868],
    "albumin": [50862],
    "bilirubin": [50885],
    "creatinine": [50912],
    "glucose": [50931],
    "hemoglobin": [51222],
    "lactate": [50813],
    "platelet count": [51265],
    "potassium": [50971],
    "sodium": [50983],
    "wbc": [51516]
}'''
LAB_ITEMIDS = {
    # Hematology
    "wbc": [51301, 51516],
    "platelets": [51265],

    # Renal / liver
    "creatinine": [50912],
    "bilirubin": [50885],

    # Metabolic
    "lactate": [50813],

    # Coagulation
    "inr": [51237],

    # Blood gas / acid–base
    "ph": [50820],
    "pco2": [50818],
    "po2": [50821],
}

ALL_LAB_ITEMIDS = sum(LAB_ITEMIDS.values(), [])

def build_labs(batch_icustay_ids, icu_meta):
    """
    Build 12-hour binned lab features from LABEVENTS.

    Returns:
        DataFrame with columns:
        icustay_id | bin | lab_1 | lab_2 | ...
    """

    chunks = pd.read_csv(
        f"{DATA_DIR}/LABEVENTS.csv",
        usecols=[
            "subject_id",
            "hadm_id",
            "itemid",
            "charttime",
            "valuenum"
        ],
        chunksize=2_000_000
    )

    out = []

    for df in chunks:
        # 1. Filter to batch stays
        df = df[df.hadm_id.isin(
            icu_meta[
                icu_meta.icustay_id.isin(batch_icustay_ids)
            ].hadm_id.unique()
        )]

        # 2. Filter to labs of interest
        df = df[df.itemid.isin(ALL_LAB_ITEMIDS)]

        if df.empty:
            continue

        # 3. Parse time
        df["charttime"] = pd.to_datetime(df["charttime"])

        # 4. Attach ICU intime
        df = df.merge(
            icu_meta[["subject_id", "hadm_id", "icustay_id", "intime"]],
            on=["subject_id", "hadm_id"],
            how="left"
        )

        # Drop labs outside ICU stay
        df = df[
            (df.charttime >= df.intime) &
            (df.charttime <= icu_meta.set_index("icustay_id")
                              .loc[df.icustay_id]
                              .outtime.values)
        ]

        # 5. Compute 12h bin
        df["bin"] = (
            (df.charttime - df.intime)
            .dt.total_seconds() // (BIN_HOURS * 3600)
        ).astype(int)

        # 6. Aggregate within bin
        df = (
            df.groupby(["icustay_id", "bin", "itemid"])["valuenum"]
            .mean()
            .reset_index()
        )

        out.append(df)

    if not out:
        return pd.DataFrame(columns=["icustay_id", "bin"])

    df = pd.concat(out, ignore_index=True)

    # 7. Pivot ITEMIDs → lab names
    lab_frames = []

    for lab, ids in LAB_ITEMIDS.items():
        tmp = (
            df[df.itemid.isin(ids)]
            .groupby(["icustay_id", "bin"])["valuenum"]
            .mean()
            .rename(lab)
        )
        lab_frames.append(tmp)

    labs = pd.concat(lab_frames, axis=1).reset_index()

    return labs


In [117]:
# 6 

FLUID_ITEMIDS = [
    41913, 6190,  # NS
    30021, 2971, 44367, 225828,  # LR
    1848,  # D5W
    3066, 1521, 2358, 40548, 46564, 45403, 227456  # Albumin
]

VASOPRESSOR_ITEMIDS = [
    30120, 221906, 30047,  # norepinephrine
    221289, 30044, 30119,  # epinephrine
    221662, 5329, 30043,  # dopamine
    221749, 5656, 30128,  # phenylephrine
    22315, 1136, 1222 # vasopressin
]

def build_inputs(batch_icustay_ids, icu_meta):
    """
    Build action features (iv_fluids, vasopressors) binned into 12-hour windows.
    """

    # 1. Load INPUTEVENTS_MV
    df = pd.read_csv(
        f"{DATA_DIR}/INPUTEVENTS_MV.csv",
        usecols=[
            "icustay_id",
            "itemid",
            "starttime",
            "endtime",
            "amount",
            "rate"
        ]
    )

    df = df[df.icustay_id.isin(batch_icustay_ids)]
    if df.empty:
        return pd.DataFrame(
            columns=["icustay_id", "bin", "iv_fluids", "vasopressors"]
        )

    df["starttime"] = pd.to_datetime(df["starttime"])
    df["endtime"] = pd.to_datetime(df["endtime"])

    # 2. Attach ICU intime
    df = df.merge(
        icu_meta[["icustay_id", "intime"]],
        on="icustay_id",
        how="left"
    )

    # 3. Build bins
    bins = []

    for _, row in icu_meta[
        icu_meta.icustay_id.isin(batch_icustay_ids)
    ].iterrows():

        n_bins = int(
            np.ceil(
                (row.outtime - row.intime).total_seconds()
                / (BIN_HOURS * 3600)
            )
        )

        for b in range(n_bins):
            bins.append({
                "icustay_id": row.icustay_id,
                "bin": b,
                "bin_start": row.intime + pd.Timedelta(hours=b * BIN_HOURS),
                "bin_end": row.intime + pd.Timedelta(hours=(b + 1) * BIN_HOURS),
            })

    bins_df = pd.DataFrame(bins)

    # 4. Helper: overlap minutes
    def overlap_minutes(start1, end1, start2, end2):
        start = max(start1, start2)
        end = min(end1, end2)
        return max(0, (end - start).total_seconds() / 60.0)

    # 5. IV FLUIDS (continuous sum)
    iv = df[df.itemid.isin(FLUID_ITEMIDS)].copy()
    iv = iv[(iv.starttime.notna()) & (iv.endtime.notna())]

    iv["duration_min"] = (
        iv.endtime - iv.starttime
    ).dt.total_seconds() / 60

    iv["rate"] = iv["rate"].fillna(
        iv["amount"] / iv["duration_min"]
    )

    iv = iv[(iv.rate.notna()) & (iv.duration_min > 0)]

    iv_values = []

    for _, bin_row in bins_df.iterrows():
        active = iv[
            (iv.icustay_id == bin_row.icustay_id) &
            (iv.endtime > bin_row.bin_start) &
            (iv.starttime < bin_row.bin_end)
        ]

        total = 0.0
        for _, r in active.iterrows():
            mins = overlap_minutes(
                r.starttime, r.endtime,
                bin_row.bin_start, bin_row.bin_end
            )
            total += mins * r.rate

        iv_values.append(total)

    bins_df["iv_fluids"] = iv_values

    # 6. VASOPRESSORS (continuous sum)
    vp = df[df.itemid.isin(VASOPRESSOR_ITEMIDS)].copy()
    vp = vp[(vp.starttime.notna()) & (vp.endtime.notna())]

    vp["duration_min"] = (
        vp.endtime - vp.starttime
    ).dt.total_seconds() / 60

    vp["rate"] = vp["rate"].fillna(
        vp["amount"] / vp["duration_min"]
    )

    vp = vp[(vp.rate.notna()) & (vp.duration_min > 0)]

    vp_values = []

    for _, bin_row in bins_df.iterrows():
        active = vp[
            (vp.icustay_id == bin_row.icustay_id) &
            (vp.endtime > bin_row.bin_start) &
            (vp.starttime < bin_row.bin_end)
        ]

        total = 0.0
        for _, r in active.iterrows():
            mins = overlap_minutes(
                r.starttime, r.endtime,
                bin_row.bin_start, bin_row.bin_end
            )
            total += mins * r.rate

        vp_values.append(total)

    bins_df["vasopressors"] = vp_values

    # 7. Final output
    return bins_df[[
        "icustay_id",
        "bin",
        "iv_fluids",
        "vasopressors"
    ]]



In [ ]:
# Backtracked this for now, I don't think Alex actually ended up using OUTPUTEVENTS or INPUTEVENTS_CV

URINE_ITEMIDS = [
    40055,  # Urine Out Foley
    40069,  # Urine Out Void
    40094,  # Urine Out Condom Cath
    40428,  # Urine Out Straight Cath
    40715,  # Urine Out Suprapubic
    40473,  # Urine Out Ileoconduit
]

def build_outputs(batch_icustay_ids, icu_meta):
    """
    Build 12-hour binned ICU output features (urine output).

    Returns:
        DataFrame with columns:
        icustay_id | bin | urine_output
    """

    chunks = pd.read_csv(
        f"{DATA_DIR}/OUTPUTEVENTS.csv",
        usecols=[
            "icustay_id",
            "itemid",
            "charttime",
            "value"
        ],
        chunksize=1_000_000
    )

    out = []

    for df in chunks:
        # 1. Filter to batch ICU stays
        df = df[df.icustay_id.isin(batch_icustay_ids)]
        if df.empty:
            continue

        # 2. Keep urine outputs only
        df = df[df.itemid.isin(URINE_ITEMIDS)]
        if df.empty:
            continue

        # 3. Parse time
        df["charttime"] = pd.to_datetime(df["charttime"])

        # 4. Attach ICU intime
        df = df.merge(
            icu_meta[["icustay_id", "intime"]],
            on="icustay_id",
            how="left"
        )

        # 5. Compute 12-hour bin
        df["bin"] = (
            (df.charttime - df.intime)
            .dt.total_seconds() // (BIN_HOURS * 3600)
        ).astype(int)

        # 6. Aggregate urine output
        urine = (
            df.groupby(["icustay_id", "bin"])["value"]
            .sum()
            .rename("urine_output")
        )

        out.append(urine)

    if not out:
        return pd.DataFrame(
            columns=["icustay_id", "bin", "urine_output"]
        )

    outputs = pd.concat(out).reset_index()
    return outputs


In [118]:
# 7

ANTIBIOTIC_KEYWORDS = [
    "cef", "ceph",
    "penicillin", "pip", "tazo",
    "vancomycin", "vanc",
    "levofloxacin", "ciprofloxacin",
    "meropenem", "imipenem",
    "azithromycin", "clarithromycin",
    "gentamicin", "amikacin",
    "clindamycin", "metronidazole",
    "doxycycline", "linezolid"
]

def build_prescriptions(batch_icustay_ids, icu_meta):
    """
    Build 12-hour binned prescription features (antibiotic exposure).

    Returns:
        DataFrame with columns:
        icustay_id | bin | antibiotics
    """

    # Map ICU stays → admissions
    batch_meta = icu_meta[
        icu_meta.icustay_id.isin(batch_icustay_ids)
    ][["subject_id", "hadm_id", "icustay_id", "intime", "outtime"]]

    hadm_ids = batch_meta.hadm_id.unique()

    chunks = pd.read_csv(
        f"{DATA_DIR}/PRESCRIPTIONS.csv",
        usecols=[
            "subject_id",
            "hadm_id",
            "startdate",
            "enddate",
            "drug"
        ],
        chunksize=500_000
    )

    out = []

    for df in chunks:
        # 1. Filter to batch admissions
        df = df[df.hadm_id.isin(hadm_ids)]
        if df.empty:
            continue

        # 2. Identify antibiotics
        drug_lower = df["drug"].str.lower().fillna("")
        is_abx = drug_lower.apply(
            lambda x: any(k in x for k in ANTIBIOTIC_KEYWORDS)
        )
        df = df[is_abx]
        if df.empty:
            continue

        # 3. Parse times
        df["startdate"] = pd.to_datetime(df["startdate"])
        df["enddate"] = pd.to_datetime(df["enddate"])

        # If enddate missing, assume one dose (common)
        df["enddate"] = df["enddate"].fillna(df["startdate"])

        # 4. Attach ICU metadata
        df = df.merge(
            batch_meta,
            on=["subject_id", "hadm_id"],
            how="left"
        )

        # Drop prescriptions outside ICU stay
        df = df[
            (df.enddate >= df.intime) &
            (df.startdate <= df.outtime)
        ]

        if df.empty:
            continue

        # 5. Expand prescription duration into bins
        rows = []

        for _, r in df.iterrows():
            start_bin = int(
                max(
                    0,
                    (r.startdate - r.intime).total_seconds()
                    // (BIN_HOURS * 3600)
                )
            )
            end_bin = int(
                (r.enddate - r.intime).total_seconds()
                // (BIN_HOURS * 3600)
            )

            for b in range(start_bin, end_bin + 1):
                rows.append({
                    "icustay_id": r.icustay_id,
                    "bin": b,
                    "antibiotics": 1
                })

        if rows:
            out.append(pd.DataFrame(rows))

    if not out:
        return pd.DataFrame(
            columns=["icustay_id", "bin", "antibiotics"]
        )

    # 6. Collapse duplicates
    prescriptions = (
        pd.concat(out, ignore_index=True)
        .groupby(["icustay_id", "bin"])["antibiotics"]
        .max()
        .reset_index()
    )

    return prescriptions


In [128]:
# 7 perform batching and binning

icu_ids = np.sort(icu_meta.icustay_id.unique())
batches = np.array_split(
    icu_ids,
    int(np.ceil(len(icu_ids) / BATCH_SIZE))
)

for i, batch_ids in enumerate(batches):
    print(f"Batch {i}: {len(batch_ids)} stays")

    batch_meta = icu_meta[
        icu_meta.icustay_id.isin(batch_ids)
    ]

    base = build_base_grid(batch_meta)
    vitals = build_vitals(batch_ids, icu_meta)
    labs = build_labs(batch_ids, icu_meta)
    inputs = build_inputs(batch_ids, icu_meta)
    #outputs = build_outputs(batch_ids, icu_meta)
    meds = build_prescriptions(batch_ids, icu_meta)

    df = base
    for feat in [vitals, labs, inputs, meds]:
        df = df.merge(
            feat,
            on=["icustay_id", "bin"],
            how="left"
        )

    df.to_parquet(
        f"{OUT_DIR}/batch_{i:02d}.parquet",
        index=False
    )


Batch 0: 136 stays


In [129]:
# 8 batch merging

files = glob.glob(f"{OUT_DIR}/batch_*.parquet")

full_df = pd.concat(
    (pd.read_parquet(f) for f in files),
    ignore_index=True
)

In [130]:
# 9 data cleaning and imputation

phys_cols = list(VITAL_ITEMIDS.keys())

full_df = full_df.sort_values(["icustay_id", "bin"])
full_df["antibiotics"] = full_df["antibiotics"].fillna(0)
full_df[phys_cols] = (
    full_df.groupby("icustay_id")[phys_cols]
    .ffill()
)

intervention_cols = []  # e.g. vasopressors, antibiotics
full_df[intervention_cols] = full_df[intervention_cols].fillna(0)

numeric_cols = full_df.select_dtypes(include="number").columns

for c in numeric_cols:
    lo = full_df[c].quantile(0.001)
    hi = full_df[c].quantile(0.999)
    full_df[c] = full_df[c].clip(lo, hi)

full_df["heart_rate"] = full_df["heart_rate"].clip(30, 220)
full_df["spo2"] = full_df["spo2"].clip(50, 100)
full_df["sbp"] = full_df["sbp"].clip(50, 180)
full_df["dbp"] = full_df["dbp"].clip(30, 140)

In [ ]:
# 10 iterative Bayesian-Ridge imputation for residual gaps

imputer = IterativeImputer(
    estimator=BayesianRidge(),
    max_iter=10,
    random_state=0
)


full_df[numeric_cols] = imputer.fit_transform(
    full_df[numeric_cols]
)

In [136]:
# 11 data audit to check validity (no Nan and no Inf), then save

assert full_df.isna().sum().sum() == 0
assert np.isfinite(full_df[numeric_cols]).all().all()

full_df.to_parquet(MERGED_PATH, index=False)